In [3]:
!pip install torch

In [1]:
import sys
print("현재 파이썬 버전:", sys.version)
print("현재 실행 경로:", sys.executable)

현재 파이썬 버전: 3.10.18 (main, Jun  5 2025, 08:13:51) [Clang 14.0.6 ]
현재 실행 경로: /opt/anaconda3/envs/torch_env/bin/python


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv("titanic/train.csv")
print(df.shape)
df.head()

(398, 9)


,mpg,cylinders,displacement,weight,acceleration,model_year
0,18.0,8,307.0,3504,12.0,70
1,15.0,8,350.0,3693,11.5,70
2,18.0,8,318.0,3436,11.0,70
3,16.0,8,304.0,3433,12.0,70
4,17.0,8,302.0,3449,10.5,70


In [12]:
# 2. X(입력)와 y(정답) 분리 (5개 컬럼 선택)
X = df.drop(['mpg'], axis=1).values # 2차원 List 형태로 변환
y = df['mpg'].values.reshape(-1, 1) # (N, 1) 형태로 변환

# 3. 학습 데이터(80%)와 테스트 데이터(20%) 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 스케일링 (학습 데이터의 분포를 기준으로 테스트 데이터도 변환)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. 텐서(Tensor)로 변환 (PyTorch 사용을 위해 필수)
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test)

In [6]:
# 1계층 모델 (단순 선형 회귀와 구조 동일)
model_1 = nn.Sequential(
    nn.Linear(5, 1)
)

# 2계층 모델 (히든 레이어 1개 추가 + 활성화 함수 ReLU)
# 강의에서 배운 수식 y = f2(W2 * f1(W1 * x)) 와 같은 구조
model_2 = nn.Sequential(
    nn.Linear(5, 32), # 은닉층 노드 32개
    nn.ReLU(),        # 활성화 함수 f1
    nn.Linear(32, 1)  # 출력층
)

# 3계층 모델 (히든 레이어 2개)
model_3 = nn.Sequential(
    nn.Linear(5, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)

In [7]:
def train_model(model, X_train, y_train, epochs=1000, lr=0.01, verbose=False):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.train() # 학습 모드 설정
    
    for epoch in range(epochs):
        # 순전파
        prediction = model(X_train)
        loss = criterion(prediction, y_train)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if verbose and (epoch+1) % 200 == 0:
            print(f'Epoch {epoch+1:4d}/{epochs} | Loss: {loss.item():.4f}')


In [10]:
def evaluate_model(model, X_test, y_test):
    model.eval() # 평가 모드 설정 (중요)
    
    with torch.no_grad(): # 미분 계산 끄기
        prediction = model(X_test)
    
    # 텐서 -> 넘파이 변환
    pred_numpy = prediction.detach().numpy()
    y_true_numpy = y_test.detach().numpy()
    
    # 지표 계산
    mse = mean_squared_error(y_true_numpy, pred_numpy)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true_numpy, pred_numpy)
    
    return mse, rmse, r2

In [11]:
models = [
    ("1계층 (Linear)", model_1),
    ("2계층 (Deep)  ", model_2),
    ("3계층 (Deeper)", model_3)
]

print(f"{'Model':<15} | {'MSE':<10} | {'RMSE':<10} | {'R2 Score':<10}")
print("-" * 55)

for name, model in models:
    # 1. 학습 (Train 데이터 사용)
    # print(f"{name} 학습 중...") 
    train_model(model, X_train_tensor, y_train_tensor, epochs=1000, lr=0.01)
    
    # 2. 평가 (Test 데이터 사용)
    mse, rmse, r2 = evaluate_model(model, X_test_tensor, y_test_tensor)
    
    # 3. 결과 출력
    print(f"{name:<15} | {mse:<10.4f} | {rmse:<10.4f} | {r2:<10.4f}")

Model           | MSE        | RMSE       | R2 Score  
-------------------------------------------------------


RuntimeError: Numpy is not available